# The Three Body Problem  
Using Verlet intergration evolve a 3 body system for position and velocity.  
Calculate angular momentum of the bodies.  
Calculate total energy (potential and kinetic).  
### Verlet  
velocity half step  
$ v_{n+\frac{1}{2}} = v_n +\frac{1}{2} ha (x_n) $  
$ x_{n+1} = x_n +hv_{n+\frac{1}{2}} $  
$ v_{n+1} = v_{n+\frac{1}{2}} +\frac{1}{2} ha (x_{n+1}) $  

$ h $ is timestep  
$ a(x) $ is acceleration at point x. I will calulate using the force function defined near the top, and divide it by mass. So that it in keeping with the other solvers here.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc
from cycler import cycler
import pandas as pd
import timeit

In [2]:
def Data_save(p, v, E, L, t, f_name):
    """Function for saving positions, velocity, energy and angular momentum produced to a txt file.
    IN:
        p: An array holding all of the positions of that timestep. Cartesain
        v: An array holding all the velocities of that timestep. Cartesain
        E: Total energy of the system
        L: Total angular momentum of the system
        t: Time of the particular step
        f_name: The file the data is being saved to."""
    f = open(f_name, "a")
    f.write("\n" + f"{p[0,0]}" + "\t" + f"{p[0,1]}" + "\t" + f"{p[1,0]}" + "\t" + f"{p[1,1]}")
    f.write("\t" + f"{v[0,0]}" + "\t" + f"{v[0,1]}" + "\t" + f"{v[1,0]}" + "\t" + f"{v[1,1]}")
    f.write("\t" + f"{E}" + "\t" + f"{L}" + "\t" +f"{t}")
    f.close()
    

In [3]:
def Grav_eng(pos_e, ma_e, pos_other, ma_other, ep, G):
    """Grav_eng(pos_e, ma_e, pos_other, ma_other):
    Calculates tthe gravtiational energy between two bodies.
    U = -GMm/(r+ep)
    IN:
        pos_e: position of the planet in question. [x,y]
        ma_e: mas of the planet in question.
        pos_other: position of the other planet. [x,y]
        ma_other: mas of the other planet.
        ep: extra bit to make sure the numbers dont blow up to much at low distances.
        G: GRavitational constant.
    OUT:
        g_e: gravitational potential energy."""
    
    dif = pos_other-pos_e
    g_e = - (G * ma_e *ma_other) / (np.linalg.norm(dif+ep))

    return g_e

def Grav_eng(ma_p, pos_p, ep, G):
    """Grav_eng(ma_p, pos_p, ep, G):
    A function calulating the gravitational potential energy of all the bodies in a system.
    U = -GMm/(r+ep)
    IN:
        ma_p: an array holding all of the massses of the planets.
        pos_p: an array holding all the postions of all of the planets.
        ep: extra bit to make sure the numbers dont blow up to much at low distances.
        G: GRavitational constant.
    OUT: the total potnetial energy of all of the bodies."""
    size = len(ma_p)
    mat = np.empty((size,size), dtype='object')
    mat[:] = 0
    for i in range(size):
        for j in range(size):
            if i==j:
                continue
            else:
                mx = ma_p[i]
                my = ma_p[j]
                dif = np.abs(pos[i]-pos[j])
                #dif = (pos[i]-pos[j])
                g_eng = -G*mx*my/ (np.linalg.norm(dif + ep))
                #print(f" {i},{j} {force}")
                mat[i,j] = mat[i,j]+g_eng
    
    mat = np.sum(mat, axis=1)
    return mat

In [4]:
def Kin_eng(vel_e, m_e):
    """Kin_eng(vel_e, m_e):
    Calculate the kinetic energy.
    E = 0.5mv^2
    IN:
        vel_e: velocity of a body. [vx,vy]
        m_e: mass of the planet in question.
    OUT:
        k_e: Kinetic energy of the body."""
    
    k_e = 0.5*m_e*vel_e**2
    k_e = np.linalg.norm(k_e)

    return k_e

In [5]:
def Angular_momentum():
    """A function which calculates the angular momentum of the system.
    IN:
    OUT:"""

In [6]:
def Force( ma_f, pos_f, ep, G):
    """Force( ma_f, pos_f, ep, G):
    A function for calculating a matricx of the gravitational force between bodies.
    F = GMm/r^2
    IN:
        ma_f: An array of all of the planetary masses.
        pos_f: position of the bodies. [x,y]
        ep: extra bit to make sure the numbers dont blow up to much at low distances.
        G: Gravitational constant.
    OUT:
        mat: a square matrix of the force between the planets"""
    size = len(ma_f)
    mat = np.zeros((size,size), dtype='object')
    mat[:] = 0
    for i in range(size):
        for j in range(size):
            
                mx = ma_f[i]
                my = ma_f[j]
                #dif = np.abs(pos[i]-pos[j])
                dif = (pos[i]-pos[j])
                force = - G*mx*my*dif / (np.linalg.norm(dif + ep)**3)
                #print(f" {i},{j} {force}")
                mat[i,j] = mat[i,j]+force
    
    mat = np.sum(mat, axis=1)
    return mat


In [7]:
def Verlet(pos_v, vel_v, ma, ep, G, t_s):
    """A function that runs a single loop of a verlet intergrator.
    IN:
        pos_v: an array holding the positions of al the planets at the beginning of a tine step.
        vel_v: an array holding the velocities at the begining of a time step.
        ma: an array holding the mass of all of the planets.
        ep: extra bit to make sure the numbers dont blow up to much at low distances.
        G: gravitational constant.
        t_s: time step.

    OUT:
        pos_full: an array holding the position of all the bodies at the end of a time step
        vel_full: an array holding the velocities of all the bodies at the end of a time step
        """

    pos_full = np.array([])
    vel_full = np.array([])
    kinetic = np.array([])
    gravitational = np.array([])
    f_half = Force(ma_f = ma, pos_f = pos_v, ep = ep, G = G)
    #print(f"f_half {f_half}")#force matrix fro half step


    for v in range(len(ma)):
        
        v_half = vel_v[v] + (0.5 * t_s * f_half[v]/ma[v])     #Caclcualting the halfstep velocities for a planet
        print(f"f_half[v]/ma[v] {f_half[v]/ma[v]}")
        p_full = pos_v[v] + (t_s * v_half)     #Caclcualting the fullstep positions for a planet

        if pos_full.shape ==(0,):
            pos_full = np.array([p_full])
        else:
            pos_full = np.vstack((pos_full, np.array([p_full])))
        #print(f" {v}")
    #print(f"pos_full {pos_full}")

    f_full = Force(ma_f = ma, pos_f = pos_full , ep = ep, G = G)     #force matrix fro full step

    for ve in range(len(ma)):
        #print(f"pos_full[ve] {pos_full[ve]}")
        v_full = v_half + (0.5 * t_s * f_full[ve]/ma[ve])     #Caclcualting the fullstep velocities for a planet
        print(f"f_full[ve]/ma[ve] {f_full[ve]/ma[ve]}")
        
        if vel_full.shape ==(0,):
            vel_full = np.array([v_full])
        else:
            vel_full= np.vstack((vel_full, np.array([v_full])))
        #print(f"ve {ve}")
      
        kinetic = np.append(kinetic, Kin_eng(v_full, ma[ve]))
        gravitational = np.append(gravitational, Grav_eng(pos_full[ve], ma[ve], pos_full, ma, ep, G))
        #print(f"ve {ve}")

    kinetic = np.sum(kinetic)
    graviational = np.sum(gravitational)
    return pos_full, vel_full, gravitational, kinetic

In [8]:
help(Kin_eng)

Help on function Kin_eng in module __main__:

Kin_eng(vel_e, m_e)
    Kin_eng(vel_e, m_e):
    Calculate the kinetic energy.
    E = 0.5mv^2
    IN:
        vel_e: velocity of a body. [vx,vy]
        m_e: mass of the planet in question.
    OUT:
        k_e: Kinetic energy of the body.



### Importing data and setting initial conditions
Initial conditions are stored in a data file

In [9]:
data = pd.read_csv("3_body_input_and_tests.txt",delim_whitespace=True)
data

C:\Users\bs583\AppData\Local\Temp\ipykernel_9532\2616778796.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv("3_body_input_and_tests.txt",delim_whitespace=True)


,name,pos1x,pos1y,pos2x,pos2y,pos3x,pos3y,vel1x,vel1y,vel2x,vel2y,vel3x,vel3y,m1,m2,m3
0,Test1,0,0,1,0,0,0.000,0.0000,0.0000,0.0000,10.0000,0.0000,0.0000,100,1,0
1,Test2,1,1,-1,-1,0,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1000,1000,0
2,Test3,1,0,-1,0,0,0.000,0.0000,5.0000,0.0000,-5.0000,0.0000,0.0000,10,10,0
3,Test4,1,3,2,2,0,0.000,3.0000,2.0000,1.0000,2.0000,0.0000,0.0000,2,2,0
4,Test5,1,1,-1,-2,3,-2.000,1.0000,1.0000,2.0000,2.0000,-1.0000,-2.0000,1,1,1
5,Test6,0,0,2,0,1,1.732,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,5,5,5
6,Test7,1,1,-2,1,4,1.000,0.0000,0.0000,0.0000,5.0000,0.0000,-5.0000,100,5,5
7,Test8,1,1,-2,1,4,1.000,1.0000,-0.5000,0.0000,5.0000,0.0000,-5.0000,100,5,5
8,FigOf8,-1,0,1,0,0,0.000,0.3471,0.5327,0.3471,0.5327,-0.6942,-1.0654,1,1,1
9,Bfly1,-1,0,1,0,0,0.000,0.3068,0.1255,0.3068,0.1255,-0.6136,0.2510,1,1,1


In [10]:
initial = 11
in_con = data.iloc[initial]

masses = np.array([in_con.loc["m1"], in_con.loc["m2"], in_con.loc["m3"]])
i_positions = np.array([[in_con.loc["pos1x"],in_con.loc["pos1y"]], [in_con.loc["pos2x"],in_con.loc["pos2y"]], [in_con.loc["pos3x"],in_con.loc["pos3y"]]])
i_velocities = np.array([[in_con.loc["vel1x"],in_con.loc["vel1y"]], [in_con.loc["vel2x"],in_con.loc["vel2y"]], [in_con.loc["vel3x"],in_con.loc["vel3y"]]])

for i in range(len(masses)):
    if masses[i] == 0:
        masses = np.delete(masses, i)
        i_positions = np.delete(i_positions, i, 0)
        i_velocities = np.delete(i_velocities, i, 0)


i_positions

array([[-1.,  0.],
       [ 1.,  0.],
       [ 0.,  0.]])

In [11]:
fin = 80
time_step = 0.001
no_steps = round(fin/time_step)
epsilon = 1e-4
G = 1
no_steps

80000

In [12]:
file_name = f"produced_data_system_{initial}.txt"
file = open(file_name, "w")

for a in range(len(masses)):
    #file.write("px1" + "\t" + "p1y" + "\t" + "p2x" + "\t" + "p2y" + "\t" + "p3x" + "\t" + "p3y" + "\t" + "vx1" + "\t" + "v1y" + "\t" + "v2x" + "\t" + "v2y" + "\t" + "v3x" + "\t" + "v3y" + "\t" + "Energy" + "\t" + "ang_mom")
    file.write(f"p{a+1}x" + "\t" + f"p{a+1}y" + "\t")
    
for a in range(len(masses)):
    file.write(f"v{a+1}x" + "\t" + f"v{a+1}y" + "\t")

file.write("Energy" + "\t" + "Ang_mom" + "\t" + "time")
    
file.close()


### running everything

In [13]:
help(Verlet)

Help on function Verlet in module __main__:

Verlet(pos_v, vel_v, ma, ep, G, t_s)
    A function that runs a single loop of a verlet intergrator.
    IN:
        pos_v: an array holding the positions of al the planets at the beginning of a tine step.
        vel_v: an array holding the velocities at the begining of a time step.
        ma: an array holding the mass of all of the planets.
        ep: extra bit to make sure the numbers dont blow up to much at low distances.
        G: gravitational constant.
        t_s: time step.

    OUT:
        pos_full: an array holding the position of all the bodies at the end of a time step
        vel_full: an array holding the velocities of all the bodies at the end of a time step



In [14]:
print(i_velocities)

[[ 0.3501  0.0793]
 [ 0.3501  0.0793]
 [-0.7002 -0.1586]]


In [ ]:
pos = i_positions
vel = i_velocities
fig =  plt.figure()
ax = fig.add_subplot()
fig_E_L, ax_E_L = plt.subplots(1,2)
ax_E_L[0].set_title("Energy")
ax_E_L[1].set_title("Angular Momentum")

t1 = timeit.default_timer()
for i in range(no_steps):
    #print(f"pos {pos[0]}")
    ax.scatter(pos[0,0], pos[0,1], color = "k")
    ax.scatter(pos[1,0] ,pos[1,1], color = "red")
    ax.scatter(pos[2,0] ,pos[2,1], color = "blue")
    time = i *time_step
    Data_save(p=pos, v=vel, E=0, L=0, t=time, f_name = file_name)


    new_pos, new_vel, gravitational_energy, kinetic_energy = Verlet(pos_v=pos, vel_v=vel, ma=masses, ep=epsilon, G=G, t_s=time_step)     #Calling verlet intergrator
    
    tot_eng = kinetic_energy + np.sum(gravitational_energy)
    ax_E_L[0].scatter(time,tot_eng, color = "Green")
    
    #print(new_pos)
    #print(f"vel-new_vel {vel-new_vel}")
    pos = new_pos
    vel = new_vel

ax.grid(True)
    
t2 = timeit.default_timer()

f_half[v]/ma[v] [1.25033755 0.        ]
f_half[v]/ma[v] [-1.24966255  0.        ]
f_half[v]/ma[v] [0.0006 0.    ]
f_full[ve]/ma[ve] [1.25033755 0.        ]
f_full[ve]/ma[ve] [-1.24966255  0.        ]
f_full[ve]/ma[ve] [0.0006 0.    ]
f_half[v]/ma[v] [ 1.25244382e+00 -2.38723269e-04]
f_half[v]/ma[v] [-1.2475676e+00 -2.3708132e-04]
f_half[v]/ma[v] [-0.00360121  0.0004758 ]
f_full[ve]/ma[ve] [ 1.25244382e+00 -2.38723269e-04]
f_full[ve]/ma[ve] [-1.2475676e+00 -2.3708132e-04]
f_full[ve]/ma[ve] [-0.00360121  0.0004758 ]
f_half[v]/ma[v] [ 1.25244696e+00 -2.38723808e-04]
f_half[v]/ma[v] [-1.24757071e+00 -2.37081850e-04]
f_half[v]/ma[v] [-0.00360124  0.0004758 ]
f_full[ve]/ma[ve] [ 1.25244696e+00 -2.38723808e-04]
f_full[ve]/ma[ve] [-1.24757071e+00 -2.37081850e-04]
f_full[ve]/ma[ve] [-0.00360124  0.0004758 ]
f_half[v]/ma[v] [ 1.25245011e+00 -2.38723992e-04]
f_half[v]/ma[v] [-1.24757381e+00 -2.37082024e-04]
f_half[v]/ma[v] [-0.00360127  0.0004758 ]
f_full[ve]/ma[ve] [ 1.25245011e+00 -2.38723992e-

In [ ]:




print(f" timetaken {t2-t1}")

In [ ]:
#ax.set_xlim(-10,10)
#ax.set_ylim(-10,10)
fig

In [ ]:
fig_E_L

In [ ]:
vel

res = pd.read_csv("produced_data_system_0.txt",delim_whitespace=True)
res

p1_posx = res["p1x"]
p1_posy = res["p1y"]

fig1 =  plt.figure()
ax1 = fig1.add_subplot()
d = np.array([p1_posx,p1_posy])
ax1.plot(d[0],d[1])

In [ ]:
help(Force)

In [ ]:
help(Grav_eng)

In [ ]:
p = np.array([[1,1], [-1,-1]])
m = np.array([100,1])
Grav_eng(m,p,0.1,1)

In [ ]:
np.linalg.norm([1,0])